In [ ]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Captura os PDFs de voos da FAB - Força Aérea Brasileira - desde 2013
#

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import os
import requests
import pandas as pd
import wget

In [12]:
# Site base para pesquisa nos anos e com todos PDFs
site = "https://www.fab.mil.br/voos"

In [13]:
# Função para pegar os links com href
def find(elem):
    element = elem.get_attribute("href")
    if element:
        return element
    else:
        return False

In [16]:
# Abre o browser e o site
# A conexão com esse site não é muito boa - tem dias que não funciona ou a página não termina de carregar
# É preciso repetir a execução se não carregar de primeira
driver = webdriver.Chrome('D:\Code\chromedriver.exe') 

driver.get(site)

In [17]:
# Lista vazia
documentos = []

In [18]:
# De 2013 a 2021 são 9 páginas no html
for i in range(1, 10):
    
    print(i)
    
    elems = driver.find_elements_by_xpath("//a[@href]")

    # Itera nos elementos encontrados
    for elem in elems:
              
        # Testa se existe um link disponível no href, esperando 1000000 segundos
        try:
            links = WebDriverWait(elem, 1000000).until(find)
        
            if links.startswith("https://www.fab.mil.br/cabine/"):
                #print(links)
                dicionario = {"link": links}
                documentos.append(dicionario)
        
        except:
            continue

    
    i += 1
    
    # Se já estiver na última sai fora do loop
    if i == 10:
        driver.close()
        break
        
    # Vai para próxima página
    pagina = "//*[@id='VoosForm_ano']/option[" + str(i) + "]"
    
    # Clica para próxima página
    element = WebDriverWait(driver, 2000).until(EC.element_to_be_clickable((By.XPATH, pagina)))
    element.click()
    time.sleep(10)
    
    # Clica em pesquisar
    element = WebDriverWait(driver, 2000).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='yt0']")))
    element.click()
    
    time.sleep(10)

1
2
3
4
5
6
7
8
9


In [19]:
# Gera dataframe com os os links de PDFs extraídos
df = pd.DataFrame(documentos)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019 entries, 0 to 2018
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    2019 non-null   object
dtypes: object(1)
memory usage: 15.9+ KB


In [ ]:
# Mostra diretório atual

In [21]:
print(os.getcwd())

D:\Code\pinpoint\repo


In [22]:
# Muda para diretório de download
os.chdir('D:\\Code\\pinpoint\\repo\\resultados\\voos_fab')

In [ ]:
# Faz o download dos PDFs
[wget.download(x) for x in df['link']]

In [ ]:
# Caso apareça no dowload erro 404 pode fazer dessa forma
# Abaixo um filtro apenas de novembro de 2021

In [26]:
nov = df[df['link'].str.contains("202111")].copy()

In [30]:
for num, row in nov.iterrows():
    link = row['link']
    print(link)
    try:
        wget.download(link)
    except:
        print('Erro')
        pass

https://www.fab.mil.br/cabine/voos/20211103_181103.pdf
100% [............................................................................] 418598 / 418598https://www.fab.mil.br/cabine/voos/20211105_111116.pdf
100% [............................................................................] 125304 / 125304https://www.fab.mil.br/cabine/voos/20211105_111104.pdf
100% [..............................................................................] 89934 / 89934https://www.fab.mil.br/cabine/voos/20211105_111128.pdf
100% [............................................................................] 103822 / 103822https://www.fab.mil.br/cabine/voos/20211110_100820.pdf
100% [............................................................................] 122953 / 122953https://www.fab.mil.br/cabine/voos/20211110_100836.pdf
100% [............................................................................] 124506 / 124506https://www.fab.mil.br/cabine/voos/20211118_111902.pdf
100% [...............